# Selecting the TensorFlow version and installing Required Packages 



In [1]:
%tensorflow_version  1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [3]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -qq Cython contextlib2 pillow lxml matplotlib
!pip install -qq pycocotools

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /content

In [ ]:
!mkdir cone_detection

Change the main directory to as per the requirement. This model was trained to detected traffic cones, hence the name

In [ ]:
%cd cone_detection/

In [ ]:
!mkdir data
!mkdir data/images data/train_labels data/test_labels

* Once all the directories are created. Upload the Images and their XML files in the appropriate directories.
* 70-30 is a good ratio for the training files vs testing



In [ ]:
!ls -1 data/train_labels/ | wc -l && ls -1 data/test_labels/ | wc -l

# Imports neccessory for the Model Training


In [11]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import pandas as pd
import numpy as np
import csv
import re
import cv2 
import os
import glob
import xml.etree.ElementTree as ET
import io
import tensorflow.compat.v1 as tf
from PIL import Image
from collections import namedtuple, OrderedDict
import shutil
import urllib.request
import tarfile
import torch
from torchvision import datasets, models, transforms
from google.colab import files

# Preprocessing Images and Labels
1. Converting the annotations from xml files to two csv files for each **train_labels** and `**train_labels**.
2. Creating a pbtxt file that specifies the number of classes (the number of different objects you want to train your model)


In [ ]:
%cd /content/cone_detection//data
# images extension
images_extension = 'jpg'

# takes the path of a directory that contains xml files and converts them to one csv file.
# returns a csv file that contains: image name, width, height, class, xmin, ymin, xmax, ymax.
# note: if the xml file contains more than one box/label, it will create more than one row for the same image. each row contains the info for an individual box. 
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text ,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

# for both the train_labels and test_labels csv files, it runs the xml_to_csv() above.
for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

/content/cone_detection/data
Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.


Creating the Label Map for all the classes. Change the **display_name** for each class

In [ ]:
label_map_path = os.path.join("label_map.pbtxt")
label_map_content = ""

#creats a pbtxt file the has the class names.
for i, class_name in enumerate(classes):
    # display_name is optional.
    label_map_content = (
        label_map_content
        + "item {{\n    id: {0}\n    name: '{1}'\n    display_name: 'cone'\n }}\n\n".format(i + 1, class_name)
    )
label_map_content = label_map_content.strip()
with open(label_map_path, "w") as f:
    f.write(label_map_content)

In [ ]:
!cat label_map.pbtxt

In [7]:
!ls 

data  models


# Downloding the TensorFlow models directory and building the Environment
1. Cloning [Tensorflow models](https://github.com/tensorflow/models.git) from the offical git repo. The repo contains the object detection API we are interseted in. 
2. Setting up the PYTHONPATH for the TensorFlow model
3. Testing the configuration by using the builder file.

In [ ]:
%cd /content/cone_detection/

In [ ]:
!git clone --q https://github.com/tensorflow/models.git

In [ ]:
%cd /content/cone_detection/models/research

In [13]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
os.environ['PYTHONPATH'] += ':/content/cone_detection/models/:/content/cone_detection/models/research/:/content/cone_detection/models/research/slim/'

In [15]:
!pip install tf_slim

In [16]:
%cd /content/cone_detection/models/research

/content/cone_detection/models/research


In [ ]:
!python setup.py build

In [ ]:
!python setup.py install

In [17]:
!python object_detection/builders/model_builder_tf1_test.py

Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params(True)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params(True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params(False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params(False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature(True)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature(True)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature(False)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature(False)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_model_from_config_with_example_miner

# Generating the TensorFlow record for the **train** and the **test** CSVs


In [ ]:
from object_detection.utils import dataset_util
%cd /content/cone_detection//models/

data_base_url = '/content/cone_detection/data/'
image_dir = data_base_url +'images/'

def class_text_to_int(row_label):
		if row_label == 'cone':
				return 1
		else:
				None

def split(df, group):
		data = namedtuple('data', ['filename', 'object'])
		gb = df.groupby(group)
		return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
		with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
				encoded_jpg = fid.read()
		encoded_jpg_io = io.BytesIO(encoded_jpg)
		image = Image.open(encoded_jpg_io)
		width, height = image.size

		filename = group.filename.encode('utf8')
		image_format = b'jpg'
		xmins = []
		xmaxs = []
		ymins = []
		ymaxs = []
		classes_text = []
		classes = []

		for index, row in group.object.iterrows():
				xmins.append(row['xmin'] / width)
				xmaxs.append(row['xmax'] / width)
				ymins.append(row['ymin'] / height)
				ymaxs.append(row['ymax'] / height)
				classes_text.append(row['class'].encode('utf8'))
				classes.append(class_text_to_int(row['class']))

		tf_example = tf.train.Example(features=tf.train.Features(feature={
				'image/height': dataset_util.int64_feature(height),
				'image/width': dataset_util.int64_feature(width),
				'image/filename': dataset_util.bytes_feature(filename),
				'image/source_id': dataset_util.bytes_feature(filename),
				'image/encoded': dataset_util.bytes_feature(encoded_jpg),
				'image/format': dataset_util.bytes_feature(image_format),
				'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
				'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
				'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
				'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
				'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
				'image/object/class/label': dataset_util.int64_list_feature(classes),
		}))
		return tf_example

for csv in ['train_labels', 'test_labels']:
  writer = tf.python_io.TFRecordWriter(data_base_url + csv + '.record')
  path = os.path.join(image_dir)
  examples = pd.read_csv(data_base_url + csv + '.csv')
  grouped = split(examples, 'filename')

  for group in grouped:
      tf_example = create_tf_example(group, path)
      writer.write(tf_example.SerializeToString())
    
  writer.close()
  output_path = os.path.join(os.getcwd(), data_base_url + csv + '.record')
  print('Successfully created the TFRecords: {}'.format(data_base_url +csv + '.record'))


/content/cone_detection/models
Successfully created the TFRecords: /content/cone_detection/data/train_labels.record
Successfully created the TFRecords: /content/cone_detection/data/test_labels.record


# Downloading the Base Model for Object Detection
1. Downloading a TensorFlow **[Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md)** model for the object detection 
2. Creating a directory to save the contents of the model

In [ ]:
%cd /content/cone_detection/models/research


/content/cone_detection/models/research


In [ ]:
!mkdir pretrained_model

In [ ]:
!ls

In [ ]:
%cd /content/cone_detection/models/research/pretrained_model

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz

In [ ]:
!tar -xvf 'ssd_mobilenet_v2_coco_2018_03_29.tar.gz'

# Configuring the Training Pipeline
1. Adding the path for the TFRecords files and pbtxt,batch_size,num_steps,num_classes to the configuration file.
2. Adding some Image augmentation.
3. Creating a directory to save the model at each checkpoint while training. 

In [ ]:
%cd /content//cone_detection/models/research/object_detection/samples/configs/

/content/cone_detection/models/research/object_detection/samples/configs


In [ ]:

%%writefile /content//cone_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config
model {
  ssd {
    num_classes: 1 #change this according to the number of objects you want to train
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.3333
      }
    }
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    box_predictor {
      convolutional_box_predictor {
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: true
        dropout_keep_probability: 0.8
        kernel_size: 1
        box_code_size: 4
        apply_sigmoid_to_scores: false
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
            weight: 0.00004
          }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.03
              mean: 0.0
            }
          }
          batch_norm {
            train: true,
            scale: true,
            center: true,
            decay: 0.9997,
            epsilon: 0.001,
          }
        }

      }
    }
    feature_extractor {
      type: 'ssd_mobilenet_v2'
      min_depth: 16
      depth_multiplier: 1.0
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            weight: 0.00004
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          train: true,
          scale: true,
          center: true,
          decay: 0.9997,
          epsilon: 0.001,
        }
      }
    }
    loss {
      classification_loss {
        weighted_sigmoid {
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000
        iou_threshold: 0.95
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 3
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
  }
}

train_config: {
  batch_size: 12
  optimizer {
    rms_prop_optimizer: {
      learning_rate: {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.003
          decay_steps: 800720
          decay_factor: 0.95
        }
      }
      momentum_optimizer_value: 0.9
      decay: 0.9
      epsilon: 1.0
    }
  }

  fine_tune_checkpoint: "/content/cone_detection/models/research/pretrained_model/model.ckpt" #path to your model.ckpt file
  fine_tune_checkpoint_type:  "detection"
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 3000 #can increase this value for better accuracy
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_adjust_contrast {
    }
  }

  data_augmentation_options {
    ssd_random_crop {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    input_path: "/content/cone_detection/data/train_labels.record" #path for your records file
  }
  label_map_path: "/content/cone_detection/data/label_map.pbtxt" #path for the label_map.pbtxt file
}

eval_config: {
  num_examples: 19 #number of images 
  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  max_evals: 10
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/content/cone_detection/data/test_labels.record"
  }
  label_map_path: "/content/cone_detection/data/label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

Overwriting /content//cone_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config


In [ ]:
!cat /content/cone_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config

In [ ]:
%cd /content/cone_detection/models/research/

/content/cone_detection/models/research


In [ ]:
!mkdir training

In [ ]:
!ls 

# Using TensorBoard to monitor Logs

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

In [ ]:
LOG_DIR = '/content/cone_detection/models/research/training/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://d5ccc1cd37c1.ngrok.io


# Training the Custom Detector 

In [ ]:
%cd /content/cone_detection/models/research/object_detection

/content/cone_detection/models/research/object_detection


In [ ]:
!python3 /content/cone_detection/models/research/object_detection/model_main.py \
--pipeline_config_path=/content/cone_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config \
--model_dir=/content/cone_detection/models/research/training   

# Exporting The Trained model



In [ ]:
!ls

In [ ]:
%cd /content/cone_detection/models/research/training/

In [ ]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('/content/cone_detection/models/research/training/model.ckpt-2000.meta')
    saver.restore(sess, '/content/cone_detection/models/research/training/model.ckpt-2000')

INFO:tensorflow:Restoring parameters from /content/cone_detection/models/research/training/model.ckpt-2000


In [ ]:
!mkdir /content/cone_detection/models/research/fine_tuned_model/

In [ ]:
!python  /content/cone_detection/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/cone_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config \
    --output_directory=/content/cone_detection/models/research/fine_tuned_model/ \
    --trained_checkpoint_prefix=/content/cone_detection/models/research/training/model.ckpt-3000 \       

## Downloading your Frozen Graph.pb and Labels.pbtxt File

In [ ]:
files.download('/content/cone_detection/models/research/fine_tuned_model/frozen_inference_graph.pb')
files.download('/content/cone_detection/data/label_map.pbtxt')

# Validating the Custom Object Classifier 

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

### Paths for the Exported graph and the labels

In [ ]:
PATH_TO_FROZEN_GRAPH = '/content/cone_detection/models/research/fine_tuned_model/frozen_inference_graph.pb'
PATH_TO_LABELS = '/content/cone_detection/data/label_map.pbtxt'
NUM_CLASSES = 1 

In [ ]:
detection_graph = tf.Graph()

with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

PATH_TO_TEST_IMAGES = '/content/cone_detection/data/images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES, '{}.jpg'.format(i)) for i in range(16, 19) ]
IMAGE_SIZE = (300, 300)

def run_inference_for_single_image(image, graph):

  with graph.as_default():
    with tf.Session() as sess:
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}

      for key in ['num_detections', 'detection_boxes', 'detection_scores', 'detection_classes', 'detection_masks' ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
          
      if 'detection_masks' in tensor_dict:

        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks( detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        tensor_dict['detection_masks'] = tf.expand_dims( detection_masks_reframed, 0)

      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
      output_dict = sess.run(tensor_dict, feed_dict={image_tensor: image})
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]

      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

for images in TEST_IMAGE_PATHS:
  
  image = Image.open(images)
  image_np = load_image_into_numpy_array(image)
  image_np_expanded = np.expand_dims(image_np, axis=0)
  output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  vis_util.visualize_boxes_and_labels_on_image_array( image_np,output_dict['detection_boxes'], output_dict['detection_classes'], output_dict['detection_scores'], 
                                                     category_index, instance_masks=output_dict.get('detection_masks'), use_normalized_coordinates=True,
                                                     line_thickness=1)
  display(Image.fromarray(image_np))
  print("\n")


# Converting the Custom Object Classifier model to TensorFlow Lite

## Exporting the graph using TFLite inference

In [18]:
%cd  /content/cone_detection/models/research/object_detection/

/content/cone_detection/models/research/object_detection


In [19]:
!mkdir tflite_model

mkdir: cannot create directory ‘tflite_model’: File exists


In [20]:
!python /content/cone_detection/models/research/object_detection/export_tflite_ssd_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/content/cone_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config \
    --output_directory=/content/cone_detection/models/research/tflite_model \
    --trained_checkpoint_prefix=/content/cone_detection/models/research/training/model.ckpt-3000 \
    --add_postprocessing_op=true

Instructions for updating:
Please use `layer.__call__` method instead.
W0831 20:19:30.855042 139987178162048 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0831 20:19:33.593176 139987178162048 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0831 20:19:33.633793 139987178162048 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0831 20:19:33.673787 139987178162048 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

In [21]:
!tflite_convert --output_file=/content/cone_detection/models/research/tflite_model/detect.tflite \
--graph_def_file=/content/cone_detection/models/research/tflite_model/tflite_graph.pb \
--inference_type=FLOAT \
--inference_input_type=FLOAT \
--input_arrays=normalized_input_image_tensor \
--input_shapes=1,300,300,3 \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--mean_values=128 --std_dev_values=128 \
--change_concat_input_ranges=false \
--allow_custom_ops

2020-08-31 20:20:21.742743: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-08-31 20:20:21.761093: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-31 20:20:21.761923: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2020-08-31 20:20:21.762201: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-08-31 20:20:21.764045: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-08-31 20:20:21.765971: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
20